In [1]:
import tensorflow as tf
# #指定使用那块GUP训练
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
config = tf.ConfigProto()
# 设置最大占有GPU不超过显存的70%
config.gpu_options.per_process_gpu_memory_fraction = 0.7 
# # 重点：设置动态分配GPU
config.gpu_options.allow_growth = True
# 创建session时
tf.Session(config=config) 

In [3]:
import numpy as np
from keras.datasets import cifar10
from torch.utils.data import Dataset
import torch.utils.data as Data
class MyDataset(Dataset):
    def __init__(self, imgs, transform=None):
        # super().__init__()
        self.imgs = imgs
        self.transform = transform

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, index):
        img = self.imgs[index]
        if self.transform is not None:
            img = self.transform(img)
        else:
            img = torch.from_numpy(img)
        img=img.reshape([3,32,32])
        return img
import torch
from torch.autograd import Variable
import model
import torch.nn.functional as F
model = model.cifar10(128)
model.load_state_dict(torch.load('./log/default/best-85.pth'))
model.cuda()
def get_mean_var(y_logit):
    y_logit=np.abs(y_logit-0.1)
    return np.mean(y_logit,axis=1)

def get_possibility(images):
    x_dataset = MyDataset(images)
    # print(x_dataset[0].shape)
    x_real_loader = Data.DataLoader(dataset=x_dataset, batch_size=100, shuffle=True)
    y_logits = []
    for i, data in enumerate(x_real_loader):
        # indx_target = target.clone()
        data = data.cuda()
        data = Variable(data, volatile=True)
        output = model(data)
        pred = F.softmax(output).cpu().detach().numpy()
        y_logits += [i for i in pred]
        y_logits=np.array(y_logits)
    return y_logits
import os
if not os.path.isdir('saved_models_{}'.format('sgan')):
    os.mkdir('saved_models_{}'.format('sgan'))
f = open('saved_models_{}/log_collapse1.txt'.format('sgan'), mode='w')
import torch.utils.data as Data
import cv2

from keras.datasets import cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import losses
from keras.utils import to_categorical
import keras.backend as K
from PIL import Image
import matplotlib.pyplot as plt
import os
import numpy as np

class SGAN:
    def __init__(self):
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(
            loss=['binary_crossentropy', 'categorical_crossentropy'],
            loss_weights=[0.5, 0.5],
            optimizer=optimizer,
            metrics=['accuracy']
        )

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        noise = Input(shape=(100,))
        img = self.generator(noise)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid, _ = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model(noise, valid)
        self.combined.compile(loss=['binary_crossentropy'], optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 8 * 8, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((8, 8, 128)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(3, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        # model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())

        model.summary()

        img = Input(shape=self.img_shape)

        features = model(img)
        valid = Dense(1, activation="sigmoid")(features)
        label = Dense(self.num_classes+1, activation="softmax")(features)

        return Model(img, [valid, label])

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (X_test, _) = cifar10.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        # X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Class weights:
        # To balance the difference in occurences of digit class labels.
        # 50% of labels that the discriminator trains on are 'fake'.
        # Weight = 1 / frequency
        half_batch = batch_size // 2
        cw1 = {0: 1, 1: 1}
        cw2 = {i: self.num_classes / half_batch for i in range(self.num_classes)}
        cw2[self.num_classes] = 1 / half_batch

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0
        steps = []
        values = []

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                imgs = X_train[index * batch_size:(index + 1) * batch_size]
                # Sample noise and generate a batch of new images
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                gen_imgs = self.generator.predict(noise)

                # One-hot encoding of labels
                labels = to_categorical(y_train[index * batch_size:(index + 1) * batch_size], num_classes=self.num_classes + 1)
                fake_labels = to_categorical(np.full((batch_size, 1), self.num_classes),
                                             num_classes=self.num_classes + 1)

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(imgs, [valid, labels], class_weight=[cw1, cw2])
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, [fake, fake_labels], class_weight=[cw1, cw2])
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                g_loss = self.combined.train_on_batch(noise, valid, class_weight=[cw1, cw2])

                # Plot the progress
                print("epoch:%d step:%d[D loss: %f, acc: %.2f%%, op_acc: %.2f%%] [G loss: %f]" % (
                    epoch, global_step, d_loss[0], 100 * d_loss[3], 100 * d_loss[4], g_loss))

                if global_step % sample_interval == 0:
                    self.mode_drop(epoch, global_step)
                   

    def mode_drop(self, epoch, global_step):
        r, c = 10, 1000
        noise = np.random.normal(0, 1, (r * c, 100))
        # sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        y_logits = get_possibility(gen_imgs)
        metrics = get_mean_var(y_logits)


        f.writelines('\n')
        f.writelines('global_step:' + str(global_step))
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in metrics)
        f.writelines('\n')


if __name__ == '__main__':
    sgan = SGAN()
    sgan.train(epochs=50, batch_size=64, sample_interval=50)



Sequential(
  (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (2): ReLU()
  (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (5): ReLU()
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (9): ReLU()
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (12): ReLU()
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): BatchNorm2d(512, eps=1e-05

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 16, 16, 32)        896       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 8, 8, 64)          18496     
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 8, 8, 64)          256       
__________

epoch:0 step:50[D loss: 0.510624, acc: 64.06%, op_acc: 17.97%] [G loss: 1.152895]


/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:41: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


epoch:0 step:51[D loss: 0.489091, acc: 69.53%, op_acc: 23.44%] [G loss: 1.192231]
epoch:0 step:52[D loss: 0.523982, acc: 62.50%, op_acc: 22.66%] [G loss: 1.034576]
epoch:0 step:53[D loss: 0.498565, acc: 64.06%, op_acc: 21.09%] [G loss: 1.202705]
epoch:0 step:54[D loss: 0.494002, acc: 67.19%, op_acc: 17.97%] [G loss: 1.295278]
epoch:0 step:55[D loss: 0.480611, acc: 67.97%, op_acc: 21.88%] [G loss: 1.144008]
epoch:0 step:56[D loss: 0.442948, acc: 75.78%, op_acc: 16.41%] [G loss: 1.158432]
epoch:0 step:57[D loss: 0.467681, acc: 70.31%, op_acc: 19.53%] [G loss: 1.278177]
epoch:0 step:58[D loss: 0.555404, acc: 54.69%, op_acc: 21.88%] [G loss: 1.058065]
epoch:0 step:59[D loss: 0.451221, acc: 74.22%, op_acc: 21.88%] [G loss: 1.167421]
epoch:0 step:60[D loss: 0.426566, acc: 81.25%, op_acc: 17.97%] [G loss: 1.252789]
epoch:0 step:61[D loss: 0.559624, acc: 58.59%, op_acc: 17.19%] [G loss: 1.143166]
epoch:0 step:62[D loss: 0.492029, acc: 69.53%, op_acc: 21.09%] [G loss: 1.090529]
epoch:0 step:63[

...

epoch:49 step:38568[D loss: 0.378658, acc: 69.53%, op_acc: 46.88%] [G loss: 1.114824]
epoch:49 step:38569[D loss: 0.448479, acc: 59.38%, op_acc: 45.31%] [G loss: 0.939643]
epoch:49 step:38570[D loss: 0.420137, acc: 60.94%, op_acc: 41.41%] [G loss: 1.509796]
epoch:49 step:38571[D loss: 0.348270, acc: 73.44%, op_acc: 41.41%] [G loss: 1.406307]
epoch:49 step:38572[D loss: 0.327750, acc: 73.44%, op_acc: 46.88%] [G loss: 1.438907]
epoch:49 step:38573[D loss: 0.305236, acc: 78.91%, op_acc: 54.69%] [G loss: 1.517604]
epoch:49 step:38574[D loss: 0.460356, acc: 47.66%, op_acc: 47.66%] [G loss: 1.181477]
epoch:49 step:38575[D loss: 0.386841, acc: 64.84%, op_acc: 44.53%] [G loss: 1.510636]
epoch:49 step:38576[D loss: 0.382413, acc: 66.41%, op_acc: 48.44%] [G loss: 1.522270]
epoch:49 step:38577[D loss: 0.461608, acc: 53.91%, op_acc: 39.84%] [G loss: 1.076920]
epoch:49 step:38578[D loss: 0.501274, acc: 53.91%, op_acc: 37.50%] [G loss: 0.923514]
epoch:49 step:38579[D loss: 0.478150, acc: 51.56%, op_

epoch:49 step:38667[D loss: 0.413992, acc: 68.75%, op_acc: 43.75%] [G loss: 1.149073]
epoch:49 step:38668[D loss: 0.357965, acc: 69.53%, op_acc: 47.66%] [G loss: 1.216363]
epoch:49 step:38669[D loss: 0.360466, acc: 70.31%, op_acc: 45.31%] [G loss: 1.213098]
epoch:49 step:38670[D loss: 0.382455, acc: 70.31%, op_acc: 47.66%] [G loss: 1.376348]
epoch:49 step:38671[D loss: 0.419323, acc: 66.41%, op_acc: 43.75%] [G loss: 1.533941]
epoch:49 step:38672[D loss: 0.321438, acc: 71.88%, op_acc: 51.56%] [G loss: 1.284014]
epoch:49 step:38673[D loss: 0.398852, acc: 67.19%, op_acc: 42.19%] [G loss: 1.227160]
epoch:49 step:38674[D loss: 0.430054, acc: 58.59%, op_acc: 44.53%] [G loss: 1.138017]
epoch:49 step:38675[D loss: 0.442110, acc: 61.72%, op_acc: 49.22%] [G loss: 1.207495]
epoch:49 step:38676[D loss: 0.400619, acc: 66.41%, op_acc: 42.19%] [G loss: 1.243005]
epoch:49 step:38677[D loss: 0.391319, acc: 65.62%, op_acc: 46.09%] [G loss: 1.148653]
epoch:49 step:38678[D loss: 0.363645, acc: 66.41%, op_

epoch:49 step:38765[D loss: 0.372280, acc: 75.00%, op_acc: 44.53%] [G loss: 1.052266]
epoch:49 step:38766[D loss: 0.391396, acc: 66.41%, op_acc: 46.88%] [G loss: 1.070060]
epoch:49 step:38767[D loss: 0.371487, acc: 71.09%, op_acc: 42.19%] [G loss: 0.992642]
epoch:49 step:38768[D loss: 0.381573, acc: 69.53%, op_acc: 44.53%] [G loss: 1.237370]
epoch:49 step:38769[D loss: 0.312902, acc: 77.34%, op_acc: 47.66%] [G loss: 1.048307]
epoch:49 step:38770[D loss: 0.357865, acc: 76.56%, op_acc: 48.44%] [G loss: 1.456851]
epoch:49 step:38771[D loss: 0.464525, acc: 55.47%, op_acc: 39.06%] [G loss: 1.560595]
epoch:49 step:38772[D loss: 0.409283, acc: 62.50%, op_acc: 41.41%] [G loss: 1.366531]
epoch:49 step:38773[D loss: 0.432362, acc: 58.59%, op_acc: 52.34%] [G loss: 1.318504]
epoch:49 step:38774[D loss: 0.421900, acc: 60.94%, op_acc: 39.06%] [G loss: 1.231101]
epoch:49 step:38775[D loss: 0.434114, acc: 55.47%, op_acc: 39.84%] [G loss: 1.232686]
epoch:49 step:38776[D loss: 0.408453, acc: 62.50%, op_

epoch:49 step:38861[D loss: 0.409318, acc: 63.28%, op_acc: 50.00%] [G loss: 1.220460]
epoch:49 step:38862[D loss: 0.477650, acc: 57.03%, op_acc: 34.38%] [G loss: 1.106472]
epoch:49 step:38863[D loss: 0.400505, acc: 65.62%, op_acc: 45.31%] [G loss: 1.378250]
epoch:49 step:38864[D loss: 0.353520, acc: 73.44%, op_acc: 42.97%] [G loss: 1.277050]
epoch:49 step:38865[D loss: 0.411419, acc: 63.28%, op_acc: 42.97%] [G loss: 1.233280]
epoch:49 step:38866[D loss: 0.346217, acc: 75.78%, op_acc: 45.31%] [G loss: 0.962394]
epoch:49 step:38867[D loss: 0.428132, acc: 58.59%, op_acc: 45.31%] [G loss: 1.026289]
epoch:49 step:38868[D loss: 0.372491, acc: 64.84%, op_acc: 42.97%] [G loss: 1.243267]
epoch:49 step:38869[D loss: 0.357023, acc: 74.22%, op_acc: 50.00%] [G loss: 1.380694]
epoch:49 step:38870[D loss: 0.382888, acc: 66.41%, op_acc: 50.78%] [G loss: 1.319686]
epoch:49 step:38871[D loss: 0.302340, acc: 80.47%, op_acc: 51.56%] [G loss: 1.242775]
epoch:49 step:38872[D loss: 0.336254, acc: 70.31%, op_

epoch:49 step:38960[D loss: 0.310583, acc: 82.81%, op_acc: 49.22%] [G loss: 1.396968]
epoch:49 step:38961[D loss: 0.469420, acc: 54.69%, op_acc: 43.75%] [G loss: 1.284338]
epoch:49 step:38962[D loss: 0.386208, acc: 69.53%, op_acc: 49.22%] [G loss: 1.466094]
epoch:49 step:38963[D loss: 0.351487, acc: 71.09%, op_acc: 49.22%] [G loss: 1.239733]
epoch:49 step:38964[D loss: 0.405589, acc: 62.50%, op_acc: 45.31%] [G loss: 1.275372]
epoch:49 step:38965[D loss: 0.404093, acc: 64.06%, op_acc: 47.66%] [G loss: 1.216494]
epoch:49 step:38966[D loss: 0.351224, acc: 74.22%, op_acc: 47.66%] [G loss: 1.353677]
epoch:49 step:38967[D loss: 0.407987, acc: 66.41%, op_acc: 42.19%] [G loss: 1.251539]
epoch:49 step:38968[D loss: 0.392580, acc: 64.84%, op_acc: 43.75%] [G loss: 1.087839]
epoch:49 step:38969[D loss: 0.419213, acc: 60.16%, op_acc: 39.06%] [G loss: 1.131809]
epoch:49 step:38970[D loss: 0.434855, acc: 58.59%, op_acc: 41.41%] [G loss: 0.963742]
epoch:49 step:38971[D loss: 0.412894, acc: 60.94%, op_